In [1]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import RidgeClassifierCV
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from aeon.transformations.collection.shapelet_based import RandomDilatedShapeletTransform
import hypergraphx as hgx
import torch 
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F 
from scipy.sparse import lil_matrix, csr_matrix

""" Recommended Versions
    numpy==1.22.4
    pandas==1.5.3
    sklearn==1.5.0
    aeon==1.1.0
    hypergraphx==1.7.8
    torch==2.2.0+cpu
    scipy==1.8.1
"""

' Recommended Versions\n    numpy==1.22.4\n    pandas==1.5.3\n    sklearn==1.5.0\n    aeon==1.1.0\n    hypergraphx==1.7.8\n    torch==2.2.0+cpu\n    scipy==1.8.1\n'

# 1. Base Function

## Hard Voting

In [2]:
def voting(testYList, 
           weightList=None, 
           p=0):
    """
    Hard voting

    """
    uniqueLabels = np.unique(testYList) 
    L = len(uniqueLabels)
    K, N = len(testYList), len(testYList[0])  
    testVY = np.zeros(N) 
    if weightList is None: 
        weightList = np.ones(K)
   
    testWeightArray = np.zeros((L, N)) 
    for i in range(N): 
        for j in range(K): 
            label_ = testYList[j, i]  
            index_ = np.arange(L)[uniqueLabels==label_] 
            testWeightArray[index_, i] += weightList[j] 
    for i in range(N):
        testVY[i] = uniqueLabels[np.argmax(testWeightArray[:, i])]
    return testVY.astype(int)

## Sereis Transformation

In [3]:
def HT(signal):
    """
    Hilbert transform.

    """
    N = len(signal) 
    rangeN = np.arange(N).astype(float) + 1.0  
    signalH = np.zeros(N)  
    for k in range(1, len(signal)+1):
        signalH[k-1] = np.sum(signal[0:k-1] / (k-rangeN)[0:k-1]) + np.sum(signal[k:] / (k-rangeN)[k:])
    return signalH

def series_transform(seriesX, 
                     mode="F"):  
    """
    Series transformation of time series sets.

    """
    seriesN, seriesD, seriesL = seriesX.shape[0], seriesX.shape[1], seriesX.shape[2]
    
    if mode == "F":  # Differential transform
        seriesFX = np.zeros((seriesN, seriesD, seriesL-1))
        for i in range(seriesN):
            for j in range(seriesD):
                seriesFX[i, j, :] = np.diff(seriesX[i, j, :], 1)
                seriesFX[i, j, :] = scale(seriesFX[i, j, :])
        return seriesFX
    
    if mode == "H":  # Hilbert transform
        seriesHX = np.zeros((seriesN, seriesD, seriesL))
        for i in range(seriesN): 
            for j in range(seriesD): 
                seriesHX[i, j, :] = HT(seriesX[i, j, :])
                seriesHX[i, j, :] = scale(seriesHX[i, j, :])
        return seriesHX    
    
    if mode == "HF":  # Differential transform after Hilbert transform
        seriesHFX = np.zeros((seriesN, seriesD, seriesL-1))
        for i in range(seriesN):  
            for j in range(seriesD):  
                seriesX_ = HT(seriesX[i, j, :]) 
                seriesHFX[i, j, :] = np.diff(seriesX_, 1)  
                seriesHFX[i, j, :] = scale(seriesHFX[i, j, :])
        return seriesHFX   

## Shapelet Hypergraph Generation

In [4]:
def shapelets_to_hypergraph_neighbors(shapelets, 
                                      neighbor_k=[3, 5], 
                                      normalize=False):
    """
    Generate hypergraphs using shapelets as nodes.

    """
    shapeletList = np.zeros_like(shapelets)
    shapeletList[::] = shapelets[::]
    n_shapelet, dim_shapelet, len_shapelet = shapeletList.shape[::]  
    
    similarity_matrix = np.zeros((n_shapelet, n_shapelet)) 
    
    if normalize:  
        for i in range(n_shapelet):
            shapeletList[i, 0, :] = scale(shapeletList[i, 0, :])

    similarity_matrix = euclidean_distances(shapeletList[:, 0, :])  # Distance matrix
    np.fill_diagonal(similarity_matrix, -np.inf)
    
    HGlist = []
    for k in range(len(neighbor_k)): 
        edges = [] 
        for i in range(len(similarity_matrix)): 
            index_ = np.argsort(similarity_matrix[i])[:neighbor_k[k]] 
            edges.append(index_)  
        HGlist.append(edges) 
        
    return HGlist

## Jaccard Similarity Matrix

In [5]:
def compute_jaccard_matrix(edges): 
    """
    Calculate the Jaccard similarity matrix for all node pairs in the hypergraph.
    """
    
    unique_nodes = np.unique(np.hstack(edges)) 
    nodes = unique_nodes.tolist() 

    num_nodes = len(nodes)
    num_edges = len(edges)
    
    node_to_idx = {node: idx for idx, node in enumerate(nodes)}
    
    H = lil_matrix((num_nodes, num_edges), dtype=int)  # Incidence matrix 
    
    for edge_id, edge in enumerate(edges):
        for node in edge:
            if node in node_to_idx:  
                node_idx = node_to_idx[node]
                H[node_idx, edge_id] = 1
    
    H = H.tocsr() 
    intersection = H.dot(H.T) 
    degrees = np.array(H.sum(axis=1)).flatten()

    intersection_dense = intersection.toarray() 
    union = degrees[:, np.newaxis] + degrees[np.newaxis, :] - intersection_dense 
    J_matrix = np.zeros((num_nodes, num_nodes), dtype=float)
    np.divide(intersection_dense, union, where=(union != 0), out=J_matrix)
    J_matrix[union == 0] = 0.0
    np.fill_diagonal(J_matrix, 1.0)
    
    return J_matrix

## Single Scale Shapelet Selection

In [6]:
def neighbor_hypergraph_based_shapelet_selection(shapeletList, 
                                                 shapeletLabels, 
                                                 neighbor_k=[3, 5],
                                                 save_rate=0.5, 
                                                 jaccard_threshold=0.9):
    """
    Select shapelets with the same length and dilation based on hypergraphs
    
    """
    n_shapelets = len(shapeletLabels) 
    
    neighbor_k = np.array(neighbor_k)
    neighbor_k = neighbor_k[neighbor_k<=n_shapelets]  
    if len(neighbor_k)==0:
        return np.arange(n_shapelets)
    
    HGlist = shapelets_to_hypergraph_neighbors(shapeletList, neighbor_k=neighbor_k)  # Shapelet hypergraphs
    edges = [] 
    for i in range(len(HGlist)):
        for j in range(len(HGlist[i])):
            edges.append(HGlist[i][j])
    nodes = np.unique(np.hstack(edges))  
    
    # Remove confusion nodes
    purityList = np.zeros(n_shapelets)
    degreeList = np.zeros(n_shapelets) 
    deleteNodeIndexes = [] 
    for i in range(len(edges)):  
        node_index_ = np.array(edges[i])  
        subLables_ = shapeletLabels[node_index_] 
        edge_size_ = len(node_index_)
        for j in range(len(node_index_)): 
            degreeList[node_index_[j]] += 1  
            purityList[node_index_[j]] += np.sum(subLables_[j]==subLables_) / edge_size_ 
    purityList = purityList / degreeList  
    
    J_matrix = compute_jaccard_matrix(edges)  # Jaccard similarity matrix
    J_matrix[J_matrix>=jaccard_threshold] = 1
    J_matrix[J_matrix<jaccard_threshold] = 0
    np.fill_diagonal(J_matrix, -1.0)

    # Shapelet selection
    uniqueLables = np.unique(shapeletLabels) 
    save_shapelet_n = int(np.ceil(save_rate*n_shapelets)) 
    saved_nodes = np.array([np.argmax(purityList)])  
    label_index = np.arange(len(uniqueLables))[uniqueLables==shapeletLabels[saved_nodes]] 
    label_index += 1
    if label_index>=len(uniqueLables):
        label_index = 0        
    sorted_nodes = np.argsort(purityList)[::-1]
    candidate_nodes = sorted_nodes[sorted_nodes!=saved_nodes[0]] 
    for i in range(save_shapelet_n-1): 
        similar_nodes_ = np.arange(n_shapelets)[J_matrix[saved_nodes[-1], :]==1]
        J_matrix[:, similar_nodes_] = -1 
        J_matrix[similar_nodes_, :] = -1
        J_matrix[saved_nodes[-1], :] = -1  
        J_matrix[:, saved_nodes[-1]] = -1
        candidate_nodes = candidate_nodes[~np.isin(candidate_nodes, similar_nodes_)]
        candidate_labels = shapeletLabels[candidate_nodes]

        if len(candidate_nodes) <= 0: 
            break    
        curr_label_ = uniqueLables[label_index]  
        if np.sum(candidate_labels==curr_label_)==0: 
            curr_label_0 = curr_label_ 
            label_index += 1
            if label_index>=len(uniqueLables):  
                label_index = 0
            curr_label_ = uniqueLables[label_index]  
            uniqueLables = uniqueLables[uniqueLables!=curr_label_0]  
        sub_candidate_nodes = candidate_nodes[candidate_labels==curr_label_]  
        if len(sub_candidate_nodes)==0:
            break
        top_node_ = sub_candidate_nodes[np.argmax(purityList[sub_candidate_nodes])]

        label_index += 1 
        if label_index>=len(uniqueLables): 
            label_index = 0          

        saved_nodes = np.hstack((saved_nodes, [top_node_]))  
        candidate_nodes = candidate_nodes[candidate_nodes!=saved_nodes[-1]]  
    return saved_nodes

## Hypergraph-based Shapelet Selection

In [7]:
def hypergraph_shapelet_selection(trainSignalX, trainY, 
                                  max_shapelets=20000, 
                                  per_shapelets=2500, 
                                  neighbor_k=[3, 5],  
                                  jaccard_threshold=0.9,  
                                  seed=0):
    """
    Select Shapelets at Different Scales Based on Hypergraphs
    """
    # Shapelets initialization
    ShapeletTransform = RandomDilatedShapeletTransform(max_shapelets=max_shapelets, random_state=seed, alpha_similarity=0.0, proba_normalization=0.8)
    ShapeletTransform.fit(trainSignalX, trainY)
    shapelets = np.zeros_like(ShapeletTransform.shapelets_[0])
    shapelets[::] = ShapeletTransform.shapelets_[0][::]
    shapeletLabels = np.zeros_like(ShapeletTransform.shapelets_[8])
    shapeletLabels[::] = ShapeletTransform.shapelets_[8][::]
    shapeletDilas = np.zeros_like(ShapeletTransform.shapelets_[3])
    shapeletDilas[::] = ShapeletTransform.shapelets_[3][::]
    shapeletNormal = np.zeros_like(ShapeletTransform.shapelets_[5]) 
    shapeletNormal[::] = ShapeletTransform.shapelets_[5][::]

    uniqueDilas = np.unique(shapeletDilas) 
    selectedShapeletIndexes = []
    save_rate = per_shapelets / len(shapelets)

    for j in range(len(uniqueDilas)):
        # Select standardized Shapelets
        shapeletIndexD_True = np.arange(len(shapeletLabels))[np.logical_and(shapeletDilas==uniqueDilas[j], shapeletNormal==1)] 
        saveRealIndexesD_True = neighbor_hypergraph_based_shapelet_selection(shapelets[shapeletIndexD_True], shapeletLabels[shapeletIndexD_True], 
                                                                         neighbor_k=neighbor_k, 
                                                                         save_rate=save_rate,
                                                                         jaccard_threshold=jaccard_threshold,
                                                                         )  
        if len(saveRealIndexesD_True)>0:
            selectedShapeletIndexes.append(shapeletIndexD_True[saveRealIndexesD_True])  

        # Select no standardized Shapelets
        shapeletIndexD_False = np.arange(len(shapeletLabels))[np.logical_and(shapeletDilas==uniqueDilas[j], shapeletNormal==0)] 
        saveRealIndexesD_False = neighbor_hypergraph_based_shapelet_selection(shapelets[shapeletIndexD_False], shapeletLabels[shapeletIndexD_False], 
                                                                         neighbor_k=neighbor_k, 
                                                                         save_rate=save_rate,
                                                                         jaccard_threshold=jaccard_threshold,
                                                                         ) 
        if len(saveRealIndexesD_False)>0:  
            selectedShapeletIndexes.append(shapeletIndexD_False[saveRealIndexesD_False]) 
    selectedShapeletIndexes = np.hstack(selectedShapeletIndexes) 

    # Update shapelets parameters
    ShapeletTransform.shapelets_ = (ShapeletTransform.shapelets_[0][selectedShapeletIndexes],
                            ShapeletTransform.shapelets_[1][selectedShapeletIndexes],
                            ShapeletTransform.shapelets_[2][selectedShapeletIndexes],
                            ShapeletTransform.shapelets_[3][selectedShapeletIndexes],
                            ShapeletTransform.shapelets_[4][selectedShapeletIndexes],
                            ShapeletTransform.shapelets_[5][selectedShapeletIndexes],
                            ShapeletTransform.shapelets_[6][selectedShapeletIndexes],
                            ShapeletTransform.shapelets_[7][selectedShapeletIndexes],
                            ShapeletTransform.shapelets_[8][selectedShapeletIndexes])    
    
    return ShapeletTransform

##  PSO Feature Transformation

In [8]:
def rdst_so_scaling(featureX, 
                    series_length, 
                    shapelet_lengths, 
                    shapelet_dilations):
    """
    Shapelet SO features transform to PSO features
    
    """
    for i in range(featureX.shape[1]): 
        length_ = shapelet_lengths[i] 
        dilation_ = shapelet_dilations[i] 
        featureX[:, i] = featureX[:, i] / (series_length - (length_-1)*dilation_)
    
    return featureX

## Ordinal Positions Conversion

In [9]:
def assign_order(positionsABS):
    """
    Convert the absolute position of Shapelets to ordinal positions
    """
    n_shapelets = len(positionsABS) 
    positionsORD = np.zeros_like(positionsABS)  
    
    unique_positions = np.unique(positionsABS)
    frontN = 0  
    for i in range(len(unique_positions)):  
        currentN = np.sum(positionsABS==unique_positions[i])
        positionsORD[positionsABS==unique_positions[i]] = frontN + (currentN-1) / 2  
        frontN = frontN + currentN 
        
    return positionsORD

## Optimization of Shapelet Position Encoding Parameters

In [10]:
class ShapeletDualPositionEncoding(nn.Module):
    """
    Optimize absolute embedding parameter Ka and ordinal embedding parameter Ko through backpropagation
    """
    def __init__(self, dim_coding=2, n_shapelet=10, len_map=100, Ka0=1.5, Ko0=1.5, K_min=1.0, K_max=10.0):
        super().__init__() 
        self.dim_coding = dim_coding 
        self.n_shapelet = n_shapelet 
        self.len_map = len_map 
        self.log_Ka = nn.Parameter(torch.tensor(np.log(Ka0)))  
        self.log_Ko = nn.Parameter(torch.tensor(np.log(Ko0))) 
        self.log_K_min = np.log(K_min)  
        self.log_K_max = np.log(K_max) 
    
    def shapelet_absolute_position_encoding(self, pos):
        log_Ka = torch.clamp(self.log_Ka, self.log_K_min, self.log_K_max)  
        base = torch.exp(log_Ka) * self.len_map / (2*np.pi)
        pe = torch.zeros(pos.size(0), self.n_shapelet * self.dim_coding) 
        pe[:, :self.n_shapelet] = torch.sin(pos / base)  
        pe[:, self.n_shapelet:] = torch.cos(pos / base)
        return pe  
    
    def shapelet_ordinal_position_encoding(self, order):
        log_Ko = torch.clamp(self.log_Ko, self.log_K_min, self.log_K_max)  
        base = torch.exp(log_Ko) * self.n_shapelet / (2*np.pi) 
        pe = torch.zeros(order.size(0), self.n_shapelet * self.dim_coding)  
        pe[:, :self.n_shapelet] = torch.sin(order / base) 
        pe[:, self.n_shapelet:] = torch.cos(order / base) 
        return pe 
    
    def forward(self, featureX, positionX, orderX):
        sape = self.shapelet_absolute_position_encoding(positionX) 
        sope = self.shapelet_ordinal_position_encoding(orderX) 
        featureX_A = featureX + sape  
        featureX_O = featureX + sope  
        return featureX_A, featureX_O 

# Similarity between feature vector sets
def cosine_similarity_squared_loss(featureX0, featureX1, eps=1e-8):
    cos_sim = F.cosine_similarity(featureX0, featureX1, dim=0, eps=eps) 
    cos_sim_sq = cos_sim ** 2  
    return cos_sim_sq.mean()

In [11]:
def position_encoding_k_optimizer(featureX, 
                                  positionX, 
                                  orderX, 
                                  dim_coding=2, 
                                  len_map=100,
                                  Ka0=8.0, 
                                  Ko0=8.0, 
                                  K_min=1.0, 
                                  K_max=15.0, 
                                  lr=0.002, 
                                  max_inter=200):
    """
    Optimization of location encoding parameters based on backpropagation
    """
    n_sample = positionX.size(0) 
    n_shapelet = positionX.size(1) 

    model = ShapeletDualPositionEncoding(dim_coding=dim_coding, n_shapelet=n_shapelet, len_map=len_map,
                                         Ka0=Ka0, Ko0=Ko0, K_max=K_max, K_min=K_min)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    objectList = np.zeros(max_inter)  
    kaList = np.zeros(max_inter)  
    koList = np.zeros(max_inter)  
    objectList[::] = np.nan
    kaList[::] = np.nan
    koList[::] = np.nan

    for epoch in range(max_inter):
        optimizer.zero_grad() 
        featureX_A, featureX_O = model(featureX, positionX, orderX)  
        
        cos_sim_sq_RA = cosine_similarity_squared_loss(featureX, featureX_A) 
        cos_sim_sq_RO = cosine_similarity_squared_loss(featureX, featureX_O)  
        cos_sim_sq_AO = cosine_similarity_squared_loss(featureX_A, featureX_O)  
        cos_loss = -(cos_sim_sq_RA + cos_sim_sq_RO + cos_sim_sq_AO) / 3 

        cos_loss.backward()
        optimizer.step()

        with torch.no_grad():
            Ka = torch.exp(model.log_Ka).item() 
            Ko = torch.exp(model.log_Ko).item()  
            if Ka < K_min or Ka > K_max or Ko < K_min or Ko > K_max:
                break
            objectList[epoch] = cos_loss.item()
            kaList[epoch] = Ka
            koList[epoch] = Ko
  
    return kaList, koList, objectList

##  Dual  Shapelet  Position  Encoding

In [12]:
class PositionalEmbedding():
    """
    Embedding absolute and ordinal positions into the original Shapelet feature vector set
    """
    def __init__(
        self,
        shapeletDilations=None,
        seriesL=None, 
        shapeletL=None, 
        learn_rate=0.002, 
        max_inter=200,                          
        baseKa0=8.0, 
        baseKr0=8.0,                          
        K_min=1.0, 
        K_max=15.0
    ):
        self.shapeletDilations = shapeletDilations
        self.seriesL = seriesL
        self.shapeletL = shapeletL
        self.learn_rate = learn_rate
        self.max_inter = max_inter
        self.baseKa0 = baseKa0
        self.baseKr0 = baseKr0
        self.K_min = K_min
        self.K_max = K_max
        
    def fit_transform(self, trainX0, trainX1, trainX_arg): 
        trainX0_ABS, trainX1_ABS = np.zeros_like(trainX0), np.zeros_like(trainX1) 
        trainX0_ABS[::], trainX1_ABS[::] = trainX0[::], trainX1[::]
        trainX0_REL, trainX1_REL = np.zeros_like(trainX0), np.zeros_like(trainX1)  
        trainX0_REL[::], trainX1_REL[::] = trainX0[::], trainX1[::]

        self.uniqueDilations = np.unique(self.shapeletDilations)
        numberPerDilations = np.zeros_like(self.uniqueDilations)  
        for i in range(len(self.uniqueDilations)):  
            numberPerDilations[i] = np.sum(self.shapeletDilations==self.uniqueDilations[i])

        self.baseABS_List = np.zeros(len(self.uniqueDilations))
        self.baseREL_List = np.zeros(len(self.uniqueDilations))  
        

        for d in range(len(self.uniqueDilations)):  # Optimizing the position encoding parameters for each dilation
            dilation_ = self.uniqueDilations[d] 
            len_map_ = self.seriesL-(self.shapeletL-1)*dilation_ 
            shapeletSubN_ = numberPerDilations[d]  
            subIndex_ = np.arange(len(self.shapeletDilations))[self.shapeletDilations==dilation_] 

            trainXd_ = torch.from_numpy(np.hstack((trainX0[:, subIndex_], trainX1[:, subIndex_])))  
            positionX_ = torch.from_numpy(trainX_arg[:, subIndex_]) 
            orderX_ = torch.zeros_like(positionX_)  
            for i in range(positionX_.size(0)): 
                orderX_[i, :] = torch.from_numpy(assign_order(positionX_[i, :].numpy()))

            kaList, krList, objectList= position_encoding_k_optimizer(trainXd_, positionX_, orderX_,
                                        dim_coding=2, len_map=len_map_,
                                        Ka0=self.baseKa0, Ko0=self.baseKr0, K_min=self.K_min, K_max=self.K_max, 
                                        lr=self.learn_rate, max_inter=self.max_inter)
            baseKa = kaList[~np.isnan(objectList)][-1] 
            baseKr = krList[~np.isnan(objectList)][-1] 

            baseABS = baseKa * len_map_ / (2*np.pi)  
            baseREL = baseKr * shapeletSubN_ / (2*np.pi) 
            self.baseABS_List[d] = baseABS
            self.baseREL_List[d] = baseREL
            
        for d in range(len(self.uniqueDilations)):  # Embedding the positions for each dilation separately
            dilation_ = self.uniqueDilations[d]  
            subIndex_ = np.arange(len(self.shapeletDilations))[self.shapeletDilations==dilation_] 
            baseABS = self.baseABS_List[d]
            baseREL = self.baseREL_List[d]

            for i in range(len(trainX_arg)):  # Absolute
                positionsABS = trainX_arg[i, subIndex_]  
                trainX0_ABS[i, subIndex_] += np.sin(positionsABS / baseABS)  
                trainX1_ABS[i, subIndex_] += np.cos(positionsABS / baseABS) 

            for i in range(len(trainX_arg)):  # Ordinal
                positionsABS = trainX_arg[i, subIndex_] 
                positionsREL = assign_order(positionsABS) 
                trainX0_REL[i, subIndex_] += np.sin(positionsREL / baseREL) 
                trainX1_REL[i, subIndex_] += np.cos(positionsREL / baseREL) 
        return trainX0_ABS, trainX1_ABS, trainX0_REL, trainX1_REL
                
    def transform(self, testX0, testX1, testX_arg): 
        testX0_ABS, testX1_ABS = np.zeros_like(testX0), np.zeros_like(testX1) 
        testX0_ABS[::], testX1_ABS[::] = testX0[::], testX1[::]
        testX0_REL, testX1_REL = np.zeros_like(testX0), np.zeros_like(testX1)
        testX0_REL[::], testX1_REL[::] = testX0[::], testX1[::]
        
        for d in range(len(self.uniqueDilations)):  # Embedding the positions for each dilation separately
            dilation_ = self.uniqueDilations[d]  
            subIndex_ = np.arange(len(self.shapeletDilations))[self.shapeletDilations==dilation_] 
            baseABS = self.baseABS_List[d]
            baseREL = self.baseREL_List[d]

            for i in range(len(testX_arg)):
                positionsABS = testX_arg[i, subIndex_] 
                testX0_ABS[i, subIndex_] += np.sin(positionsABS / baseABS) 
                testX1_ABS[i, subIndex_] += np.cos(positionsABS / baseABS) 

            for i in range(len(testX_arg)):
                positionsABS = testX_arg[i, subIndex_]
                positionsREL = assign_order(positionsABS)  
                testX0_REL[i, subIndex_] += np.sin(positionsREL / baseREL) 
                testX1_REL[i, subIndex_] += np.cos(positionsREL / baseREL) 
        return testX0_ABS, testX1_ABS, testX0_REL, testX1_REL

## Multiview Feature Fusion

In [13]:
class CombinationFusion():
    """
    Paired view feature vector fusion
    """

    def __init__(
        self
    ):
        ()
    def fit_transform(self, trainX_, trainX_ABS, trainX_ORD, trainY): 
        trainX_O_A, trainX_O_R, trainX_A_R = np.zeros_like(trainX_), np.zeros_like(trainX_), np.zeros_like(trainX_)

        self.ldaList_O_A = [] 
        self.ldaList_O_R = [] 
        self.ldaList_A_R = [] 
        self.ldaFalgs_O_A = np.zeros(trainX_.shape[1])
        self.ldaFalgs_O_R = np.zeros(trainX_.shape[1])
        self.ldaFalgs_A_R = np.zeros(trainX_.shape[1])
        for i in range(trainX_.shape[1]):
            # Fusion of original and absolute embedded features
            lda = LinearDiscriminantAnalysis(n_components=1) 
            try:
                lda.fit(np.hstack((trainX_[:, i:i+1], trainX_ABS[:, i:i+1])), trainY) 
                trainX_F = lda.transform(np.hstack((trainX_[:, i:i+1], trainX_ABS[:, i:i+1]))) 
                trainX_O_A[:, i:i+1] = trainX_F[::]
                self.ldaFalgs_O_A[i] = 1  
            except:  
                trainX_F = np.mean(np.hstack((trainX_[:, i:i+1], trainX_ABS[:, i:i+1])), axis=1, keepdims=True) 
                trainX_O_A[:, i:i+1] = trainX_F[::]
            self.ldaList_O_A.append(lda)  
            
            # Fusion of original and ordinal embedded features
            lda = LinearDiscriminantAnalysis(n_components=1) 
            try:
                lda.fit(np.hstack((trainX_[:, i:i+1], trainX_ORD[:, i:i+1])), trainY) 
                trainX_F = lda.transform(np.hstack((trainX_[:, i:i+1], trainX_ORD[:, i:i+1])))  
                trainX_O_R[:, i:i+1] = trainX_F[::]
                self.ldaFalgs_O_R[i] = 1 
            except: 
                trainX_F = np.mean(np.hstack((trainX_[:, i:i+1], trainX_ORD[:, i:i+1])), axis=1, keepdims=True)  
                trainX_O_R[:, i:i+1] = trainX_F[::]
            self.ldaList_O_R.append(lda)  

            # Fusion of absolute and ordinal embedded features
            lda = LinearDiscriminantAnalysis(n_components=1)  
            try:
                lda.fit(np.hstack((trainX_ABS[:, i:i+1], trainX_ORD[:, i:i+1])), trainY) 
                trainX_F = lda.transform(np.hstack((trainX_ABS[:, i:i+1], trainX_ORD[:, i:i+1])))  
                trainX_A_R[:, i:i+1] = trainX_F[::]
                self.ldaFalgs_A_R[i] = 1 
            except:  
                trainX_F = np.mean(np.hstack((trainX_ABS[:, i:i+1], trainX_ORD[:, i:i+1])), axis=1, keepdims=True) 
                trainX_A_R[:, i:i+1] = trainX_F[::]
            self.ldaList_A_R.append(lda)  

        self.threshold = 20 
        trainX_O_A[trainX_O_A>self.threshold] = self.threshold
        trainX_O_R[trainX_O_R>self.threshold] = self.threshold
        trainX_A_R[trainX_A_R>self.threshold] = self.threshold
        
        # Feature standardization
        self.scaler_O_A = MinMaxScaler(feature_range=(-1, 1))
        self.scaler_O_A.fit(trainX_O_A)
        trainX_O_A = self.scaler_O_A.transform(trainX_O_A)
        self.scaler_O_R = MinMaxScaler(feature_range=(-1, 1))
        self.scaler_O_R.fit(trainX_O_R)
        trainX_O_R = self.scaler_O_R.transform(trainX_O_R)
        self.scaler_A_R = MinMaxScaler(feature_range=(-1, 1))
        self.scaler_A_R.fit(trainX_A_R)
        trainX_A_R = self.scaler_A_R.transform(trainX_A_R)
        
        return trainX_O_A, trainX_O_R, trainX_A_R

    def transform(self, testX_, testX_ABS, testX_ORD): 
        testX_O_A, testX_O_R, testX_A_R = np.zeros_like(testX_), np.zeros_like(testX_), np.zeros_like(testX_) 

        for i in range(testX_.shape[1]):
            # Fusion of original and absolute embedded features
            lda = self.ldaList_O_A[i]
            if self.ldaFalgs_O_A[i]==1:         
                testX_F = lda.transform(np.hstack((testX_[:, i:i+1], testX_ABS[:, i:i+1]))) 
                testX_O_A[:, i:i+1] = testX_F[::]
            else:
                testX_F = np.mean(np.hstack((testX_[:, i:i+1], testX_ABS[:, i:i+1])), axis=1, keepdims=True)  
                testX_O_A[:, i:i+1] = testX_F[::]

            # Fusion of original and ordinal embedded features
            lda = self.ldaList_O_R[i]
            if self.ldaFalgs_O_R[i]==1:                
                testX_F = lda.transform(np.hstack((testX_[:, i:i+1], testX_ORD[:, i:i+1])))  
                testX_O_R[:, i:i+1] = testX_F[::]
            else:
                testX_F = np.mean(np.hstack((testX_[:, i:i+1], testX_ORD[:, i:i+1])), axis=1, keepdims=True) 
                testX_O_R[:, i:i+1] = testX_F[::]

            # Fusion of absolute and ordinal embedded features
            lda = self.ldaList_A_R[i]
            if self.ldaFalgs_A_R[i]==1: 
                testX_F = lda.transform(np.hstack((testX_ABS[:, i:i+1], testX_ORD[:, i:i+1])))
                testX_A_R[:, i:i+1] = testX_F[::]
            else:
                testX_F = np.mean(np.hstack((testX_ABS[:, i:i+1], testX_ORD[:, i:i+1])), axis=1, keepdims=True) 
                testX_A_R[:, i:i+1] = testX_F[::]

        testX_O_A[testX_O_A>self.threshold] = self.threshold
        testX_O_R[testX_O_R>self.threshold] = self.threshold
        testX_A_R[testX_A_R>self.threshold] = self.threshold
        
        # Feature standardization
        testX_O_A = self.scaler_O_A.transform(testX_O_A)
        testX_O_R = self.scaler_O_R.transform(testX_O_R)
        testX_A_R = self.scaler_A_R.transform(testX_A_R)
        
        return testX_O_A, testX_O_R, testX_A_R
        

## Multiview Feature Extraction and Fusion

In [14]:
class MultiviewFusion():
    """
    Multiview shapelet feature extraction and fusion
    """
    def __init__(
        self,
        ShapeletTransform=None,
        baseKa0=8.0, 
        baseKr0=8.0, 
        K_min=1.0, 
        K_max=15.0,
        learn_rate=0.002, 
        max_inter=200,
        seed=0
    ):
        self.ShapeletTransform = ShapeletTransform
        self.baseKa0 = baseKa0
        self.baseKr0 = baseKr0
        self.K_min = K_min
        self.K_max = K_max
        self.learn_rate = learn_rate
        self.max_inter = max_inter
        self.seed = seed
        
    def fit_transform(self, trainSignalX, trainY): 
        trainX = self.ShapeletTransform.transform(trainSignalX)  # Initial feature extraction
        trainX[np.isnan(trainX)] = 0
        trainX[np.isinf(trainX)] = 0
        
        self.seriesL = trainSignalX.shape[2] 
        shapeletLengths = self.ShapeletTransform.shapelets_[2]  
        shapeletL = shapeletLengths[0]  
        shapeletDilations = self.ShapeletTransform.shapelets_[3] 
        
        trainX[:, 2::3] = rdst_so_scaling(trainX[:, 2::3], self.seriesL, shapeletLengths, shapeletDilations)[::] 
        trainX_min, trainX_arg, trainX_soo = trainX[:, 0::3], trainX[:, 1::3], trainX[:, 2::3]
        
        # Feature standardization
        self.scalerMIN = MinMaxScaler(feature_range=(-1, 1))
        self.scalerMIN.fit(trainX_min)
        trainX_min = self.scalerMIN.transform(trainX_min)

        self.scalerSOO = MinMaxScaler(feature_range=(-1, 1))
        self.scalerSOO.fit(trainX_soo)
        trainX_soo = self.scalerSOO.transform(trainX_soo)
     
        # Dual shapelet position encoding
        self.positional_embedding = PositionalEmbedding(shapeletDilations=shapeletDilations, 
                                                        seriesL=self.seriesL, 
                                                        shapeletL=shapeletL, 
                                                        learn_rate=self.learn_rate,
                                                        max_inter=self.max_inter,
                                                        baseKa0=self.baseKa0, 
                                                        baseKr0=self.baseKr0,
                                                        K_min=self.K_min, 
                                                        K_max=self.K_max)

        trainX_minABS, trainX_sooABS, trainX_minREL, trainX_sooREL = self.positional_embedding.fit_transform(trainX_min, trainX_soo, trainX_arg)

        # MIN feature fusion
        self.combination_fusion_min = CombinationFusion()
        trainX_minO_A, trainX_minO_R, trainX_minA_R = self.combination_fusion_min.fit_transform(trainX_min, trainX_minABS, trainX_minREL, trainY)
        # PSO feature fusion
        self.combination_fusion_soo = CombinationFusion()
        trainX_sooO_A, trainX_sooO_R, trainX_sooA_R = self.combination_fusion_soo.fit_transform(trainX_soo, trainX_sooABS, trainX_sooREL, trainY)
        
        return [trainX_min, trainX_soo, 
                trainX_minO_A, trainX_minO_R, trainX_minA_R,
                trainX_sooO_A, trainX_sooO_R, trainX_sooA_R]
        
    def transform(self, testSignalX):   
        testX = self.ShapeletTransform.transform(testSignalX)  # Initial feature extraction
        testX[np.isnan(testX)] = 0
        testX[np.isinf(testX)] = 0
        
        shapeletLengths = self.ShapeletTransform.shapelets_[2]  
        shapeletDilations = self.ShapeletTransform.shapelets_[3] 
                
        testX[:, 2::3] = rdst_so_scaling(testX[:, 2::3], self.seriesL, shapeletLengths, shapeletDilations)[::]  
        testX_min, testX_arg, testX_soo = testX[:, 0::3], testX[:, 1::3], testX[:, 2::3]
        
        # Feature standardization
        testX_min = self.scalerMIN.transform(testX_min)
        testX_soo = self.scalerSOO.transform(testX_soo)
        
        # Dual shapelet position encoding
        testX_minABS, testX_sooABS, testX_minREL, testX_sooREL = self.positional_embedding.transform(testX_min, testX_soo, testX_arg)
        
        # MIN feature fusion
        testX_minO_A, testX_minO_R, testX_minA_R = self.combination_fusion_min.transform(testX_min, testX_minABS, testX_minREL)
        # PSO feature fusion
        testX_sooO_A, testX_sooO_R, testX_sooA_R = self.combination_fusion_soo.transform(testX_soo, testX_sooABS, testX_sooREL)

        return [testX_min, testX_soo, 
                testX_minO_A, testX_minO_R, testX_minA_R, 
                testX_sooO_A, testX_sooO_R, testX_sooA_R]      

# 2. Shapeleter Classifier

In [15]:
""" ShapeleterClassifier
    A novel shapelet-based TSC algorithm Shapelet Enhancer (Shapeleter).

"""

__maintainer__ = ["Changchun He"]
__all__ = ["ShapeleterClassifier"]

class ShapeleterClassifier():
    """
    Shapelet Enhancer (Shapeleter) for Time Series Classification.

    Parameters
    ----------
    max_shapelets : int, default=20000
        The number of shapelet candidates per series representation
    per_shapelets : int, default=2500
        The number of selected shapelets per series representation
    neighbor_k : list, default=[3, 5]
        The sizes of shapelet hyperedge
    jaccard_threshold : float, default=0.9
        The Jaccard similarity threshold
    baseKa0 : float, default=8.0
        The frequency factor for SAPE
    baseKr0 : float, default=8.0
        The frequency factor for SOPE
    K_max : float, default=15.0
        The maximum of frequency factor
    K_min : float, default=1.0
        The minimum of frequency factor
    learn_rate : float, default=0.002
        The learning rate
    max_inter : int, default=200
        The maximum of iterations for positional encoding optimization
    random_state: int, default=None
        Controls randomness
    """
    def __init__(
        self,
        max_shapelets=20000,  
        per_shapelets=2500,
        neighbor_k=[3, 5],  
        jaccard_threshold=0.9,  
        baseKa0=8.0,
        baseKr0=8.0, 
        K_max=15.0, 
        K_min=1.0,
        learn_rate=0.002,  
        max_inter=200, 
        random_state=None,
    ):
        self.max_shapelets = max_shapelets
        self.per_shapelets = per_shapelets
        self.neighbor_k = neighbor_k
        self.jaccard_threshold = jaccard_threshold
        self.baseKa0 = baseKa0
        self.baseKr0 = baseKr0
        self.K_max = K_max
        self.K_min = K_min
        self.learn_rate = learn_rate
        self.max_inter = max_inter
        self.random_state = random_state
        
    def fit(self, trainSignalX, trainY): 
        """Fit a pipeline on cases (trainSignalX, trainY), where trainY is the target variable.

        Parameters
        ----------
        trainSignalX : 3D np.ndarray of shape = [n_cases, n_channels, n_timepoints]
            The training data.
        trainY : array-like, shape = [n_cases]
            The class labels. Each type of label is int.

        Returns
        -------
        self :
            Reference to self.
        """
        # series transform
        trainSignalT0X = series_transform(trainSignalX, mode="F")
        trainSignalT1X = series_transform(trainSignalX, mode="H")
        trainSignalT2X = series_transform(trainSignalX, mode="HF")
        
        # Shapelet selection
        self.ShapeletTransformRaw = hypergraph_shapelet_selection(trainSignalX, 
                                                                  trainY, 
                                                                  max_shapelets=self.max_shapelets,
                                                                  per_shapelets=self.per_shapelets,
                                                                  neighbor_k=self.neighbor_k,
                                                                  jaccard_threshold=self.jaccard_threshold,
                                                                  seed=self.random_state)
        self.ShapeletTransformT0 = hypergraph_shapelet_selection(trainSignalT0X, 
                                                                 trainY, 
                                                                 max_shapelets=self.max_shapelets,
                                                                 per_shapelets=self.per_shapelets,
                                                                 neighbor_k=self.neighbor_k,
                                                                 jaccard_threshold=self.jaccard_threshold,
                                                                 seed=self.random_state)
        self.ShapeletTransformT1 = hypergraph_shapelet_selection(trainSignalT1X, 
                                                                 trainY, 
                                                                 max_shapelets=self.max_shapelets,
                                                                 per_shapelets=self.per_shapelets,
                                                                 neighbor_k=self.neighbor_k,
                                                                 jaccard_threshold=self.jaccard_threshold,
                                                                 seed=self.random_state)
        self.ShapeletTransformT2 = hypergraph_shapelet_selection(trainSignalT2X, 
                                                                 trainY, 
                                                                 max_shapelets=self.max_shapelets,
                                                                 per_shapelets=self.per_shapelets,
                                                                 neighbor_k=self.neighbor_k,
                                                                 jaccard_threshold=self.jaccard_threshold,
                                                                 seed=self.random_state)  
        # Feature extraction and fusion
        self.multiview_fusionRaw = MultiviewFusion(self.ShapeletTransformRaw, 
                                                   baseKa0=self.baseKa0, 
                                                   baseKr0=self.baseKr0,
                                                   K_min=self.K_min, 
                                                   K_max=self.K_max,
                                                   learn_rate=self.learn_rate, 
                                                   max_inter=self.max_inter,
                                                   seed=self.random_state)
        [trainX_minORI, trainX_sooORI, 
         trainX_minO_A, trainX_minO_R, trainX_minA_R,
         trainX_sooO_A, trainX_sooO_R, trainX_sooA_R] = self.multiview_fusionRaw.fit_transform(trainSignalX, trainY)

        self.multiview_fusionT0 = MultiviewFusion(self.ShapeletTransformT0, 
                                                  baseKa0=self.baseKa0, 
                                                  baseKr0=self.baseKr0,
                                                  K_min=self.K_min, 
                                                  K_max=self.K_max,
                                                  learn_rate=self.learn_rate, 
                                                  max_inter=self.max_inter,
                                                  seed=self.random_state)
        [trainT0X_minORI, trainT0X_sooORI, 
         trainT0X_minO_A, trainT0X_minO_R, trainT0X_minA_R,
         trainT0X_sooO_A, trainT0X_sooO_R, trainT0X_sooA_R] = self.multiview_fusionT0.fit_transform(trainSignalT0X, trainY)     

        self.multiview_fusionT1 = MultiviewFusion(self.ShapeletTransformT1, 
                                                  baseKa0=self.baseKa0, 
                                                  baseKr0=self.baseKr0,
                                                  K_min=self.K_min, 
                                                  K_max=self.K_max,
                                                  learn_rate=self.learn_rate, 
                                                  max_inter=self.max_inter,
                                                  seed=self.random_state)
        [trainT1X_minORI, trainT1X_sooORI, 
         trainT1X_minO_A, trainT1X_minO_R, trainT1X_minA_R,
         trainT1X_sooO_A, trainT1X_sooO_R, trainT1X_sooA_R] = self.multiview_fusionT1.fit_transform(trainSignalT1X, trainY)         

        self.multiview_fusionT2 = MultiviewFusion(self.ShapeletTransformT2, 
                                                  baseKa0=self.baseKa0, 
                                                  baseKr0=self.baseKr0,
                                                  K_min=self.K_min, 
                                                  K_max=self.K_max,
                                                  learn_rate=self.learn_rate, 
                                                  max_inter=self.max_inter,
                                                  seed=self.random_state)
        [trainT2X_minORI, trainT2X_sooORI, 
         trainT2X_minO_A, trainT2X_minO_R, trainT2X_minA_R,
         trainT2X_sooO_A, trainT2X_sooO_R, trainT2X_sooA_R] = self.multiview_fusionT2.fit_transform(trainSignalT2X, trainY)        
        
        # Classifiers training
        # View 0
        self.clf_V0 = RidgeClassifierCV(np.logspace(-4, 4, 20))
        self.clf_V0.fit(np.hstack((trainX_minORI, trainX_sooORI,
                                   trainT0X_minORI, trainT0X_sooORI,
                                   trainT1X_minORI, trainT1X_sooORI,
                                   trainT2X_minORI, trainT2X_sooORI)), trainY)
        # View 1
        self.clf_V1 = RidgeClassifierCV(np.logspace(-4, 4, 20)) 
        self.clf_V1.fit(np.hstack((trainX_minORI, trainX_sooORI,   trainX_minO_A, trainX_sooO_A,
                                   trainT0X_minORI, trainT0X_sooORI,   trainT0X_minO_A, trainT0X_sooO_A,
                                   trainT1X_minORI, trainT1X_sooORI,   trainT1X_minO_A, trainT1X_sooO_A,
                                   trainT2X_minORI, trainT2X_sooORI,   trainT2X_minO_A, trainT2X_sooO_A)), trainY)
        # View 2
        self.clf_V2 = RidgeClassifierCV(np.logspace(-4, 4, 20))  
        self.clf_V2.fit(np.hstack((trainX_minORI, trainX_sooORI,   trainX_minO_R, trainX_sooO_R,
                                   trainT0X_minORI, trainT0X_sooORI,   trainT0X_minO_R, trainT0X_sooO_R,
                                   trainT1X_minORI, trainT1X_sooORI,   trainT1X_minO_R, trainT1X_sooO_R,
                                   trainT2X_minORI, trainT2X_sooORI,   trainT2X_minO_R, trainT2X_sooO_R)), trainY)
        # View 3
        self.clf_V3 = RidgeClassifierCV(np.logspace(-4, 4, 20)) 
        self.clf_V3.fit(np.hstack((trainX_minORI, trainX_sooORI,   trainX_minA_R, trainX_sooA_R,
                                   trainT0X_minORI, trainT0X_sooORI,   trainT0X_minA_R, trainT0X_sooA_R,
                                   trainT1X_minORI, trainT1X_sooORI,   trainT1X_minA_R, trainT1X_sooA_R,
                                   trainT2X_minORI, trainT2X_sooORI,   trainT2X_minA_R, trainT2X_sooA_R)), trainY)
        
        
    def predict(self, testSignalX):
        """Predict class values of n instances in testSignalX.

        Parameters
        ----------
        testSignalX : 3D np.ndarray of shape = [n_cases, n_channels, n_timepoints]
            The data to make predictions for testSignalX.

        Returns
        -------
        y : array-like, shape = [n_cases]
            Predicted class labels.
        """
        # series transform
        testSignalT0X = series_transform(testSignalX, mode="F")
        testSignalT1X = series_transform(testSignalX, mode="H")
        testSignalT2X = series_transform(testSignalX, mode="HF")
        
        # Feature extraction and fusion
        [testX_minORI, testX_sooORI, 
         testX_minO_A, testX_minO_R, testX_minA_R, 
         testX_sooO_A, testX_sooO_R, testX_sooA_R] = self.multiview_fusionRaw.transform(testSignalX)
        
        [testT0X_minORI, testT0X_sooORI, 
         testT0X_minO_A, testT0X_minO_R, testT0X_minA_R, 
         testT0X_sooO_A, testT0X_sooO_R, testT0X_sooA_R] = self.multiview_fusionT0.transform(testSignalT0X)  
        
        [testT1X_minORI, testT1X_sooORI, 
         testT1X_minO_A, testT1X_minO_R, testT1X_minA_R, 
         testT1X_sooO_A, testT1X_sooO_R, testT1X_sooA_R] = self.multiview_fusionT1.transform(testSignalT1X)    
        
        [testT2X_minORI, testT2X_sooORI, 
         testT2X_minO_A, testT2X_minO_R, testT2X_minA_R, 
         testT2X_sooO_A, testT2X_sooO_R, testT2X_sooA_R] = self.multiview_fusionT2.transform(testSignalT2X)    
        
        # Ensemble classification
        # View 0
        testPY_V0 = self.clf_V0.predict(np.hstack((testX_minORI, testX_sooORI,
                                                   testT0X_minORI, testT0X_sooORI,
                                                   testT1X_minORI, testT1X_sooORI,
                                                   testT2X_minORI, testT2X_sooORI)))
        # View 1
        testPY_V1 = self.clf_V1.predict(np.hstack((testX_minORI, testX_sooORI,   testX_minO_A, testX_sooO_A,
                                                    testT0X_minORI, testT0X_sooORI,   testT0X_minO_A, testT0X_sooO_A,
                                                    testT1X_minORI, testT1X_sooORI,   testT1X_minO_A, testT1X_sooO_A,
                                                    testT2X_minORI, testT2X_sooORI,   testT2X_minO_A, testT2X_sooO_A)))
        # View 2
        testPY_V2 = self.clf_V2.predict(np.hstack((testX_minORI, testX_sooORI,   testX_minO_R, testX_sooO_R,
                                                   testT0X_minORI, testT0X_sooORI,   testT0X_minO_R, testT0X_sooO_R,
                                                   testT1X_minORI, testT1X_sooORI,   testT1X_minO_R, testT1X_sooO_R,
                                                   testT2X_minORI, testT2X_sooORI,   testT2X_minO_R, testT2X_sooO_R)))
        # View 3
        testPY_V3 = self.clf_V3.predict(np.hstack((testX_minORI, testX_sooORI,   testX_minA_R, testX_sooA_R,
                                                   testT0X_minORI, testT0X_sooORI,   testT0X_minA_R, testT0X_sooA_R,
                                                   testT1X_minORI, testT1X_sooORI,   testT1X_minA_R, testT1X_sooA_R,
                                                   testT2X_minORI, testT2X_sooORI,   testT2X_minA_R, testT2X_sooA_R)))
        # Hard voting
        testPY = voting(np.vstack((testPY_V0, testPY_V1, testPY_V2, testPY_V3)))
        return testPY

## Classification Example

In [16]:
# Loading time series set
from aeon.datasets import load_arrow_head
trainSeriesX, trainY = load_arrow_head("TRAIN")
testSeriesX, testY = load_arrow_head("TEST")
# Note that the input and output labels of our classifier are in the format of int.
trainY, testY = trainY.astype(int), testY.astype(int)

# Classification
accList = np.zeros(10)
for seed in range(10):
    shapeleter = ShapeleterClassifier(random_state=seed)
    shapeleter.fit(trainSeriesX, trainY)
    testPY = shapeleter.predict(testSeriesX)
    acc = np.sum(testPY==testY) / len(testY)
    print('seed %d: %0.4f'%(seed, acc))
    accList[seed] = acc

seed 0: 0.8857
seed 1: 0.8743
seed 2: 0.8800
seed 3: 0.8743
seed 4: 0.8800
seed 5: 0.8686
seed 6: 0.8800
seed 7: 0.8743
seed 8: 0.8800
seed 9: 0.8686


In [17]:
print("Accuracy Results", np.round(accList, 4))
print("Mean Accuracy %0.4f"%np.mean(accList))

Accuracy Results [0.8857 0.8743 0.88   0.8743 0.88   0.8686 0.88   0.8743 0.88   0.8686]
Mean Accuracy 0.8766
